# Decision Tree Implementation - ID3

### Requirements:

#### Diturunkan Sendiri:
- Program bisa membuat sebuah objek pohon yang bisa menyimpan attributes dari tree (v)
- Objek pohon dapat membuat decision tree dari data yang diberikan, dan menyimpan atribut-atribut dari pohon tersebut (v)
- Objek pohon dapat menyimpan node-node yang merupakan splitting points untuk membuat keputusan (v)
- Objek pohon dapat mengakses seluruh node yang ada pada pohon
- Objek pohon dapat memilih splitting point untuk tiap keadaan; apakah menggunakan metrik information gain atau gain ratio (v)
- Objek pohon dapat mempertimbangkan atribut yang value-nya continuous dan diskrit (v)
- Objek pohon dapat mempertimbangkan atribut yang mempunyai missing value (v)
- Objek pohon dapat melakukan post-pruning dengan menggunakan 20% data untuk validasi. Detil pruning kurang lebih: https://www.quora.com/How-can-I-find-a-real-step-by-step-example-of-a-decision-tree-pruning-to-overcome-overfitting
- Objek pohon dapat menampilkan pohon yang dibuat
- Objek node dapat melakukan splitting pada dataset (menentukan keputusan harus ke node mana setelah suatu kondisi)
    - Objek node tahu harus melakukan splitting pada atribut apa
    - Objek node menyimpan splitting points pada atribut yang bersangkutan

#### Dari Spek:
- Overfitting training data dengan post pruning. Gunakanlah 20% training data untuk data validasi.
- Continuous-valued attribute: information gain dari kandidate. (v)
- Alternative measures for selecting attributes: gain ratio. (v)
- Handling missing attribute value: most common target value. (v)
- full-training the data 
- menampilkan modelnya.

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv("play_tennis.csv")
data.head()
proportion = data['play'].value_counts()/len(data)
print(proportion[0])
entropy = 0
for p in proportion.tolist():
    print(p)
    entropy -= p*math.log(p,2)
    
print(data[data['outlook'] == 'sunny'])

0.6428571428571429
0.642857142857
0.357142857143
Empty DataFrame
Columns: [day, outlook, temp, humidity, wind, play]
Index: []


In [3]:
#read iris data
load, target = load_iris(return_X_y=True)
iris_data = pd.DataFrame(load, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_data['label'] = pd.Series(target)

In [4]:
#definisi kelas Node
#Node merupakan split point pada tree. 
#Kelas ini menyimpan data yang ada pada suatu split point, atribut apa yang digunakan untuk splitting, dan tipe atribut tsb. Atau jika node merupakan daun maka disimpan value-nya
#Kelas ini dapat menentukan splitting point kebawah dari suatu node, baik atribut splittingnya kontinu maupun diskrit
#Atribut-atribut Node: 
# - data: subset data
# - split_attr: nama atribut yang akan di split
# - split_values: value cabang dari node (merupakan satu integer jika continuous, dan multiple values jika categorical)
# - target_attr: atribut label/atribut target prediksi
# - attr_cont_split: splitting point dari atribut tsb (jika atribut tsb kontinu)
# - is_leaf: apakah node merupakan daun atau tidak
# - leaf_value: nilai hasil prediksi jika node merupakan daun
# - childs: anak dari node yang berupa node
class Node:
    #konstruktor
    def __init__(self, data, split_attr, target_attr, is_continuous=False, split_value_continuous=None, is_leaf=False, leaf_value=None, parent_value=None):
        self.data = data
        self.split_attr = split_attr
        self.target_attr = target_attr
        self.childs = []
        self.is_leaf = is_leaf
        self.split_values = [split_value_continuous]
        self.leaf_value = leaf_value
        self.parent_value = parent_value

    #check apakah split attribute == numerik
    def is_attr_categorical(self):
        return self.data[self.split_attr].dtype == 'O'
    
    #get splits node jika node bukan daun
    def get_splits(self):
        if( not self.check_if_leaf()):
            #jika atribut split categorical
            if(self.is_attr_categorical()):
                #tentukan split values
                self.split_values = self.data[self.split_attr].unique()
            #jika atribut numerik / continuous, split value sudah didefinisikan sejak konstruksi objek
            return self.split_values
                        
    #add a child to a node
    def add_child(self, node):
        self.childs.append(node)

In [41]:
#definisi kelas Tree
#Kelas ini mengkonstruksi decision tree dengan menghubungkan sekumpulan node, juga memilih untuk tiap node 
#atribut apa yang akan digunakan untuk splitting. Kelas ini dapat mempertimbangkan atribut yang mengandung nilai null.
#Metrik yang  digunakan bisa dipilih antara information gain atau gain ratio.
#Kelas ini dapat melakukan pruning pada tree yang dibuat, dan juga dapat mencetak model tree yang telah dibuat
#NOTE: Asumsi missing value, bernilai "None" atau "none"
#Atribut-atribut Tree:
# - data: merupakan data yang digunakan untuk training
# - target_attr: atribut yang menjadi target prediksi (label)
# - root: node yang merupakan root
# - use_info_gain: True/False. Jika true maka metrik pemilihan atribut menggunakan information gain. Jika False, metrik menggunakan gain ratio
class Tree:
    #konstruktor
    def __init__(self, data, target_attr, use_info_gain=True,root_value=None):
        self.data = data
        self.target_attr = target_attr
        self.root = None
        self.root_value = root_value
        self.use_info_gain = use_info_gain
    
    #cari entropi total pada data
    def total_entropy(self, data):
        proportion = data[self.target_attr].value_counts()/len(data)
        print('proportion', proportion)
        entropy = 0
        for p in proportion.tolist():
            entropy -= p*math.log(p,2)
        print('entropy =', entropy)
        return entropy
    
    #hitung information gain dari suatu kolom
    def info_gain(self, kolom):
        data = self.data
        data_entropy = self.total_entropy(data)
        proportion_kolom = data[kolom].value_counts()/len(data)
        sum_entropy_kolom = 0
        for value_kolom, value_proportion in zip(proportion_kolom.index.tolist(), proportion_kolom.tolist()):
            #print("here checking")
            #print(data[data[kolom] == value_kolom])
            entropy_value_kolom = self.total_entropy(data[data[kolom] == value_kolom])
            sum_entropy_kolom -= value_proportion*entropy_value_kolom
            print('    value entropy kolom for', kolom, ':', value_kolom, ':', value_proportion, '=', entropy_value_kolom )
        print('sum entropy kolom for', kolom, '=', sum_entropy_kolom, ' data entropy =', data_entropy, ' info gain = ', data_entropy + sum_entropy_kolom )
        return data_entropy + sum_entropy_kolom
    
    #hitung information split pada data di suatu atribut
    def split_info(self, attr):
        proportion = self.data[attr].value_counts()/len(data)
        split_info = 0
        for p in proportion.tolist():
            split_info -= p*math.log(p,2)
        return split_info
    
    #hitung gain ratio untuk suatu atribut
    def gain_ratio(self, attr):
        return info_gain(attr)/split_info(attr)
    
    #cari split-split yang memungkinkan pada atribut continuous
    def find_possible_splits_continuous(self, sorted_data, split_attr):
        sorted_target = sorted_data[self.target_attr].values.tolist()
        sorted_attr = sorted_data[split_attr].values.tolist()
        prev_target_value = sorted_target[0]
        possible_splits = []
        #iterasi target value, cari titik-titik dimana 
        try:
            for i in range(1, len(sorted_target)):
                el = sorted_target[i]
                if (prev_target_value != el):
                    possible_splits.append(0.5*(sorted_attr[i] + sorted_attr[i-1]))
                prev_target_value = el
        except Exception as e:
            print(e)
        finally:
            return possible_splits
    
    #cari gain dari tiap split dan cari split optimum
    def find_optimum_split_continuous(self, pos_splits, sorted_data, split_attr):
        optimum_split = 0
        max_info_gain = -1
        #iterate split
        for i, el in enumerate(pos_splits):
            #hitung information gain
            current_gain = self.calculate_info_gain_continuous(el, sorted_data, split_attr)
            #jika information gain lebih dari sebelumnya, ganti optimum split
            if(current_gain > max_info_gain):
                max_info_gain = current_gain
                optimum_split = el
        print('optimum split for', split_attr, '=', optimum_split)
        return optimum_split
    
    #cari information gain pada suatu split continuous
    def calculate_info_gain_continuous(self, split_value, sorted_data, split_attr):
        data_entropy = self.total_entropy(sorted_data)
        #pisah data mjd "<=" dan ">" split_value
        data_less_than_equal = sorted_data[sorted_data[split_attr] <= split_value]
        data_more_than = sorted_data[sorted_data[split_attr] > split_value]
        print('len data 1', len(data_less_than_equal))
        print('len data 2', len(data_more_than))
        print('len data 3', len(sorted_data))
        #hitung entropi kolom
        entropy_less_than_equal = (float(len(data_less_than_equal))/len(sorted_data)) * self.total_entropy(data_less_than_equal)
        entropy_more_than = (float(len(data_more_than))/len(sorted_data)) * self.total_entropy(data_more_than)
#         print('>=', len(data_less_than_equal)/len(sorted_data), self.total_entropy(data_less_than_equal))
#         print('<', len(data_more_than)/len(sorted_data), self.total_entropy(data_less_than_equal))
        print('entropy_> =', entropy_more_than, 'entropy_<= =', entropy_less_than_equal)
        return data_entropy - entropy_less_than_equal - entropy_more_than
    
    #check apakah attribute == numerik
    def is_attr_categorical(self, attr):
        return self.data[attr].dtype == 'O'
    
    #handling missing value
    def handle_missing_value(self, split_attr):
        if(self.is_attr_categorical(split_attr)):
            mode = self.data[split_attr].mode().values[0]
            self.data[split_attr] = self.data[split_attr].replace({None:mode})        
    
    #buat tree
    def make_tree(self):
        print('--------------')
        #cari info_gain dari masing-masing kolom 
        data_X = self.data.drop(self.target_attr, axis=1)
        
        #basis-1: jika data terbagi dg sempurna
        if(self.data[self.target_attr].nunique() == 1):
            self.root = Node("none", "none", "none", is_leaf=True, leaf_value=self.data[self.target_attr].unique()[0], parent_value=self.root_value)
            return self.root
        
        #basis-2: jika tidak ada atribut
        if(len(data_X.columns) == 0):
            self.root = Node("none", "none", "none", is_leaf=True, leaf_value=self.data[self.target_attr].mode().values[0], parent_value=self.root_value)
            return self.root
        
        #rekurens, jika data tidak bisa mjd leaf
        else:
            max_metric = -1
            split_attr = ""
            is_split_attr_categorical = True
            for attr in data_X.columns:
                #Jika kolom kategorikal
                if(self.is_attr_categorical(attr)):
                    if(self.use_info_gain):
                        current_metric = self.info_gain(attr)
                    else:
                        current_metric = self.gain_ratio(attr)
                #jika kolom numerik
                else:
                    #sort data
                    sorted_data = self.data.sort_values(by=attr)
                    #cari split-split yang memungkinkan 
                    pos_splits = self.find_possible_splits_continuous(sorted_data, attr)
                    #hitung gain dari tiap continuous split dan cari nilai optimum
                    split_value_continuous = self.find_optimum_split_continuous(pos_splits, sorted_data, attr)
                    #hitung gain ketika sudah diketahui nilai optimum
                    current_metric = self.calculate_info_gain_continuous(split_value_continuous, sorted_data, attr)

                print(attr, ":", current_metric)
                #jika ditemukan maximum info gain di kolom tertentu
                if(current_metric > max_metric):
                    max_metric = current_metric
                    split_attr = attr
                    is_split_attr_categorical = self.is_attr_categorical(attr)
                    if (not is_split_attr_categorical):
                        split_value_attr = split_value_continuous
            
            #setelah atribut dipilih, cek apakah ada missing value
            #impute missing value dengan modus pada atribut tsb. (asumsi: atribut yg di handle hanyalah kategorikal)
            self.handle_missing_value(split_attr)
            
            #buat node
            #jika atribut terpilih == kategorikal
            if(is_split_attr_categorical):
                self.root = Node(self.data, split_attr, self.target_attr, parent_value=self.root_value)
                split_values = self.data[split_attr].unique()
                #iterate all split values
                for split_value in split_values:
                    filtered_data = self.data[self.data[split_attr] == split_value].drop(split_attr, axis=1)
                    self.root.add_child(Tree(filtered_data, self.target_attr, root_value = split_value).make_tree())

            #jika atribut terpilih == numerik & kontinu
            else:
                self.root = Node(self.data, split_attr, self.target_attr, is_continuous=True, split_value_continuous=split_value_attr)
                #filter <=
                filtered_data = self.data[self.data[split_attr] <= split_value_attr].drop(split_attr, axis=1)
                self.root.add_child(Tree(filtered_data, self.target_attr).make_tree())

                #filter >
                filtered_data = self.data[self.data[split_attr] > split_value_attr].drop(split_attr, axis=1)
                self.root.add_child(Tree(filtered_data, self.target_attr).make_tree())

            return self.root

    def print_tree(self, node, depth, space):
        if (depth == 0):
            dash = ''
        else:
            dash = '|' + '-'*space + '>'
            
        if(node.is_leaf):
            output = ('|' + (' '*space))*(depth-1) + dash + '{' + str(node.leaf_value) + '}'
        else:
            output = ('|' + (' '*space))*(depth-1) + dash + node.split_attr 
        
        if (node.parent_value):
            output = output + '    (' + node.parent_value + ')'
        
        print(output)
        
        depth += 1
        for child in node.childs:
            self.print_tree(child, depth, space)
            
    #bagian rekursif untuk prediksi
    def get_prediction_result(self, prediction_instance, node):
        #basis - jika node merupakan leaf, kembalikan value
        if(node.is_leaf):
            return node.leaf_value
        
        #rekurens - jika node bukan leaf, cari anaknya yang tepat, telusuri anak
        else:
            for child in node.childs:
                if (child.parent_value == prediction_instance[node.split_attr]):
                    return self.get_prediction_result(prediction_instance, child)
                    break
    
    #prediksi suatu dataset test
    def predict(self, test_data):
        pred_result = []
        #iterasi seluruh instance pada test_data
        for i in range(len(test_data)):
            #instance untuk di prediksi
            prediction_instance = test_data.iloc[i]
            #get prediction untuk instance yang dicek, lalu append ke hasil
            pred_result.append(self.get_prediction_result(prediction_instance, self.root))
        return pred_result

In [42]:
tree_iris = Tree(iris_data, 'label')
root_iris = tree_iris.make_tree()
tree_iris.print_tree(root_iris, 0, 2)

--------------
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 16)
('len data 2', 134)
('len data 3', 150)
('proportion', 0    1.0
Name: label, dtype: float64)
('entropy =', 0.0)
('proportion', 2    0.373134
1    0.373134
0    0.253731
Name: label, dtype: float64)
('entropy =', 1.5634074323981904)
('entropy_> =', 1.3966439729423834, 'entropy_<= =', 0.0)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 22)
('len data 2', 128)
('len data 3', 150)
('proportion', 0    0.909091
2    0.045455
1    0.045455
Name: label, dtype: float64)
('entropy =', 0.5304060778306042)
('proportion', 2    0.382812
1    0.382812
0    0.234375
Name: label, dtype: float64)
('entropy =', 1.5511852922535474)
('entropy_> =', 1.3236781160563604, 'entropy_<= =', 0.07779289141515529)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: labe

('proportion', 0    0.561798
1    0.337079
2    0.101124
Name: label, dtype: float64)
('entropy =', 1.330465426809402)
('proportion', 2    0.672131
1    0.327869
Name: label, dtype: float64)
('entropy =', 0.9127341558073343)
('entropy_> =', 0.3711785566949826, 'entropy_<= =', 0.7894094865735786)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 89)
('len data 2', 61)
('len data 3', 150)
('proportion', 0    0.561798
1    0.337079
2    0.101124
Name: label, dtype: float64)
('entropy =', 1.330465426809402)
('proportion', 2    0.672131
1    0.327869
Name: label, dtype: float64)
('entropy =', 0.9127341558073343)
('entropy_> =', 0.3711785566949826, 'entropy_<= =', 0.7894094865735786)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 89)
('len data 2', 61)
('len data 3', 150)
('proportion', 0    0.561798
1    0.337079
2    0.10

('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 8)
('len data 2', 142)
('len data 3', 150)
('proportion', 1    0.750
2    0.125
0    0.125
Name: label, dtype: float64)
('entropy =', 1.061278124459133)
('proportion', 2    0.345070
0    0.345070
1    0.309859
Name: label, dtype: float64)
('entropy =', 1.5831516551150238)
('entropy_> =', 1.4987169001755558, 'entropy_<= =', 0.05660149997115376)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 19)
('len data 2', 131)
('len data 3', 150)
('proportion', 1    0.684211
2    0.263158
0    0.052632
Name: label, dtype: float64)
('entropy =', 1.1050139971135324)
('proportion', 0    0.374046
2    0.343511
1    0.282443
Name: label, dtype: float64)
('entropy =', 1.5753776105088115)
('entropy_> =', 1.375829779844362, 'entropy_<= =', 0.1399684396343808)
('proportion', 2    0.333333
1 

('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 83)
('len data 2', 67)
('len data 3', 150)
('proportion', 1    0.506024
2    0.397590
0    0.096386
Name: label, dtype: float64)
('entropy =', 1.3516377723645527)
('proportion', 0    0.626866
2    0.253731
1    0.119403
Name: label, dtype: float64)
('entropy =', 1.290504114965777)
('entropy_> =', 0.5764251713513804, 'entropy_<= =', 0.7479062340417192)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 83)
('len data 2', 67)
('len data 3', 150)
('proportion', 1    0.506024
2    0.397590
0    0.096386
Name: label, dtype: float64)
('entropy =', 1.3516377723645527)
('proportion', 0    0.626866
2    0.253731
1    0.119403
Name: label, dtype: float64)
('entropy =', 1.290504114965777)
('entropy_> =', 0.5764251713513804, 'entropy_<= =', 0.7479062340417192)
('proportion', 2    0.33

('len data 1', 144)
('len data 2', 6)
('len data 3', 150)
('proportion', 2    0.347222
1    0.347222
0    0.305556
Name: label, dtype: float64)
('entropy =', 1.5824207639595793)
('proportion', 0    1.0
Name: label, dtype: float64)
('entropy =', 0.0)
('entropy_> =', 0.0, 'entropy_<= =', 1.5191239334011961)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 144)
('len data 2', 6)
('len data 3', 150)
('proportion', 2    0.347222
1    0.347222
0    0.305556
Name: label, dtype: float64)
('entropy =', 1.5824207639595793)
('proportion', 0    1.0
Name: label, dtype: float64)
('entropy =', 0.0)
('entropy_> =', 0.0, 'entropy_<= =', 1.5191239334011961)
('optimum split for', 'sepal_width', '=', 3.3)
('proportion', 2    0.333333
1    0.333333
0    0.333333
Name: label, dtype: float64)
('entropy =', 1.584962500721156)
('len data 1', 113)
('len data 2', 37)
('len data 3', 150)
('proportion', 1    0.433628
2    0.398230

Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 18)
('len data 2', 82)
('len data 3', 100)
('proportion', 1    0.888889
2    0.111111
Name: label, dtype: float64)
('entropy =', 0.5032583347756457)
('proportion', 2    0.585366
1    0.414634
Name: label, dtype: float64)
('entropy =', 0.9788698505067785)
('entropy_> =', 0.8026732774155584, 'entropy_<= =', 0.09058650025961623)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 18)
('len data 2', 82)
('len data 3', 100)
('proportion', 1    0.888889
2    0.111111
Name: label, dtype: float64)
('entropy =', 0.5032583347756457)
('proportion', 2    0.585366
1    0.414634
Name: label, dtype: float64)
('entropy =', 0.9788698505067785)
('entropy_> =', 0.8026732774155584, 'entropy_<= =', 0.09058650025961623)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 24)
('len data 2', 76)
('len data 3', 100)
('proportion', 1    0.875
2    0.125
Name: lab

('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 80)
('len data 2', 20)
('len data 3', 100)
('proportion', 1    0.5875
2    0.4125
Name: label, dtype: float64)
('entropy =', 0.9777945702913884)
('proportion', 2    0.85
1    0.15
Name: label, dtype: float64)
('entropy =', 0.6098403047164004)
('entropy_> =', 0.12196806094328008, 'entropy_<= =', 0.7822356562331108)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 80)
('len data 2', 20)
('len data 3', 100)
('proportion', 1    0.5875
2    0.4125
Name: label, dtype: float64)
('entropy =', 0.9777945702913884)
('proportion', 2    0.85
1    0.15
Name: label, dtype: float64)
('entropy =', 0.6098403047164004)
('entropy_> =', 0.12196806094328008, 'entropy_<= =', 0.7822356562331108)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 80)
('len data 2', 20)
('len data 3', 100)
('proportion', 1    0.5875
2    0.412

('proportion', 1    0.618182
2    0.381818
Name: label, dtype: float64)
('entropy =', 0.9593160320543777)
('proportion', 2    0.644444
1    0.355556
Name: label, dtype: float64)
('entropy =', 0.9389320105807949)
('entropy_> =', 0.4225194047613577, 'entropy_<= =', 0.5276238176299077)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 55)
('len data 2', 45)
('len data 3', 100)
('proportion', 1    0.618182
2    0.381818
Name: label, dtype: float64)
('entropy =', 0.9593160320543777)
('proportion', 2    0.644444
1    0.355556
Name: label, dtype: float64)
('entropy =', 0.9389320105807949)
('entropy_> =', 0.4225194047613577, 'entropy_<= =', 0.5276238176299077)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 55)
('len data 2', 45)
('len data 3', 100)
('proportion', 1    0.618182
2    0.381818
Name: label, dtype: float64)
('entropy =', 0.9593160320543777)
('proportion', 2    0.644444
1    0.355556
Name:

('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 52)
('len data 2', 48)
('len data 3', 100)
('proportion', 1    0.923077
2    0.076923
Name: label, dtype: float64)
('entropy =', 0.39124356362925566)
('proportion', 2    0.958333
1    0.041667
Name: label, dtype: float64)
('entropy =', 0.24988229283318547)
('entropy_> =', 0.11994350055992903, 'entropy_<= =', 0.20344665308721296)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 52)
('len data 2', 48)
('len data 3', 100)
('proportion', 1    0.923077
2    0.076923
Name: label, dtype: float64)
('entropy =', 0.39124356362925566)
('proportion', 2    0.958333
1    0.041667
Name: label, dtype: float64)
('entropy =', 0.24988229283318547)
('entropy_> =', 0.11994350055992903, 'entropy_<= =', 0.20344665308721296)
('proportion', 2    0.5
1    0.5
Name: label, dtype: float64)
('entropy =', 1.0)
('len data 1', 52)
('len data 2', 48)
('len data 3', 100)
('propo

('proportion', 1    0.75
2    0.25
Name: label, dtype: float64)
('entropy =', 0.8112781244591328)
('proportion', 1    0.938776
2    0.061224
Name: label, dtype: float64)
('entropy =', 0.3322866302216151)
('entropy_> =', 0.3072083939784743, 'entropy_<= =', 0.06122853769502889)
('proportion', 1    0.924528
2    0.075472
Name: label, dtype: float64)
('entropy =', 0.3860189005698934)
('len data 1', 4)
('len data 2', 49)
('len data 3', 53)
('proportion', 1    0.75
2    0.25
Name: label, dtype: float64)
('entropy =', 0.8112781244591328)
('proportion', 1    0.938776
2    0.061224
Name: label, dtype: float64)
('entropy =', 0.3322866302216151)
('entropy_> =', 0.3072083939784743, 'entropy_<= =', 0.06122853769502889)
('proportion', 1    0.924528
2    0.075472
Name: label, dtype: float64)
('entropy =', 0.3860189005698934)
('len data 1', 15)
('len data 2', 38)
('len data 3', 53)
('proportion', 1    0.866667
2    0.133333
Name: label, dtype: float64)
('entropy =', 0.5665095065529053)
('proportion', 

In [7]:
data_X = data.drop('day', axis=1)
tree = Tree(data_X, 'play', use_info_gain=True)
root = tree.make_tree()

print('-------tree-------')
tree.print_tree(root, 0, 2)
print('-------predict-------')
print(tree.predict(data_X.tail(4)))

--------------
('    value entropy kolom for', 'outlook', ':', 'Rain', ':', 0.35714285714285715, '=', 0.9709505944546686)
('    value entropy kolom for', 'outlook', ':', 'Sunny', ':', 0.35714285714285715, '=', 0.9709505944546686)
('    value entropy kolom for', 'outlook', ':', 'Overcast', ':', 0.2857142857142857, '=', 0.0)
('sum entropy kolom for', 'outlook', '=', -0.6935361388961918, ' data entropy =', 0.9402859586706309, ' info gain = ', 0.2467498197744391)
('outlook', ':', 0.2467498197744391)
('    value entropy kolom for', 'temp', ':', 'Mild', ':', 0.42857142857142855, '=', 0.9182958340544896)
('    value entropy kolom for', 'temp', ':', 'Cool', ':', 0.2857142857142857, '=', 0.8112781244591328)
('    value entropy kolom for', 'temp', ':', 'Hot', ':', 0.2857142857142857, '=', 1.0)
('sum entropy kolom for', 'temp', '=', -0.9110633930116763, ' data entropy =', 0.9402859586706309, ' info gain = ', 0.029222565658954647)
('temp', ':', 0.029222565658954647)
('    value entropy kolom for',

In [8]:
#hitung entropi total dataset
def total_entropy(data):
    proportion = data['play'].value_counts()/len(data)
    entropy = 0
    for p in proportion.tolist():
        entropy -= p*math.log(p,2)
    return entropy

#hitung information gain dari suatu kolom
def gain(data, kolom):
    data_entropy = total_entropy(data)
    print('KOLOM:', kolom.upper())
    print('total entropy of current data', '=',data_entropy)
    proportion_kolom = data[kolom].value_counts()/len(data)
    sum_entropy_kolom = 0
    for value_kolom, value_proportion in zip(proportion_kolom.index.tolist(), proportion_kolom.tolist()):
        entropy_value_kolom = total_entropy(data[data[kolom] == value_kolom])
        sum_entropy_kolom -= value_proportion*entropy_value_kolom
        print('value entropy kolom for', kolom, ':', value_kolom, ':', value_proportion, '=', entropy_value_kolom )
    print('sum entropy kolom for', kolom, '=', sum_entropy_kolom)
    return data_entropy + sum_entropy_kolom

#get current_data
def get_node_data(data, kolom, value):
    new_data = data[data[kolom] == value]
    return new_data.drop(kolom, axis=1)

#get current_columns
def get_current_columns(data):
    return data.drop('play', axis=1).columns

### Iterasi 1

In [9]:
current_data = data
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['play'].value_counts().index.tolist())

('KOLOM:', 'DAY')
('total entropy of current data', '=', 0.9402859586706309)
('value entropy kolom for', 'day', ':', 'D1', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D3', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D2', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D5', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D4', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D7', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D6', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D9', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D8', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D13', ':', 0.07142857142857142, '=', 0.0)
('value entropy kolom for', 'day', ':', 'D12', ':', 0.07142857142857142, '=', 0.0)
('value entropy kol

In [10]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


#### Root = Ada Hangout

### Iterasi 2

#### ada hangout = ya

In [11]:
current_data = get_node_data(data, 'ada hangout', 'ya')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [ ]:
current_data

#### ada hangout = tidak

In [ ]:
current_data = get_node_data(data, 'ada hangout', 'tidak')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data

### Iterasi 3

#### ada hangout = tidak ^ deadline = urgent

In [ ]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'urgent')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data

#### ada hangout = tidak ^ deadline = dekat

In [ ]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data

#### ada hangout = tidak ^ deadline = tidak ada

In [ ]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'tidak ada')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data

### Iterasi 4

#### ada hangout = tidak ^ deadline = dekat ^ malas = tidak

In [ ]:
current_data = get_node_data(get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat'), 'malas', 'tidak')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data

#### ada hangout = tidak ^ deadline = dekat ^ malas = ya

In [ ]:
current_data = get_node_data(get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat'), 'malas', 'ya')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

In [ ]:
current_data